# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

# Testing: Historical Daily Weather from One Call API
---

In [8]:
url_hist_daily = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658362065&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_daily_req = requests.get(url_hist_daily)
print(f'Status: {hist_daily_req.status_code}')

wd_hist = hist_daily_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 429
dict_keys(['cod', 'message'])


KeyError: 'data'

In [7]:
wd_hist

{'cod': 429,
 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription http://openweathermap.org/price'}

In [ ]:
url_hist_point = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658363457&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_point_req = requests.get(url_hist_point)
print(f'Status: {hist_point_req.status_code}')

wd_hist = hist_point_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

## Function hist_weather(unix_datetime)

In [ ]:
def hist_weather(unix_datetime, lat, lon):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid=d43015e86fe9a04995afe46ef9ad42c4"
    hist_point_req = requests.get(url_hist_point)
    print(f'API Status: {hist_point_req.status_code}')

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    # Convert dt UNIX to datetime UTC
    wd_hist_df['datetime'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_hist_df

# Testing: Current/7-Day Forecast Daily Weather from One Call API

In [ ]:
url_current_daily = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_daily_req = requests.get(url_current_daily)
print(f'Status: {current_daily_req.status_code}')

wd_current = current_daily_req.json()
print(wd_current.keys())

wd_current_df = json_normalize(wd_current['daily'])
wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# TEMPERATURE - KELVINS
temp_list = []
for i in range(len(wd_current['daily'])):
    avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
    temp_list.append(avg_temp)
wd_current_df['temp_avg'] = temp_list

# FEELS LIKE - KELVINS
fl_list = []
for i in range(len(wd_current['daily'])):
    avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
    fl_list.append(avg_fl)
wd_current_df['feels_like_avg'] = fl_list

# Convert dt UNIX to datetime UTC
wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_current_df.head(10)

## Function curr_weather(lat, lon):

In [ ]:
def curr_weather(lat, lon):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid=d43015e86fe9a04995afe46ef9ad42c4"
    current_daily_req = requests.get(url_current_daily)
    print(f'Status: {current_daily_req.status_code}')

    wd_current = current_daily_req.json()
    print(wd_current.keys())

    wd_current_df = json_normalize(wd_current['daily'])
    wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_current['daily'])):
        avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_current_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_current['daily'])):
        avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_current_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_current_df

# Hourly - Need Paid Subscription for Historical Hourly

## Testing: Current/2-Day Forecast Hourly Weather from One Call API
---

In [ ]:
url_current_hourly = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,daily&appid=d43015e86fe9a04995afe46ef9ad42c4"
current_hourly_req = requests.get(url_current_hourly)
print(f'Status: {current_hourly_req.status_code}')

wh_current = current_hourly_req.json()
print(wh_current.keys())

wh_current_df = json_normalize(wh_current['hourly'])
wh_current_df = wh_current_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# Convert dt UNIX to datetime UTC
wh_current_df['datetime'] = wh_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wh_current_df.head()

## Testing: Historical Hourly Weather from History API

In [ ]:
'''
url_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=d43015e86fe9a04995afe46ef9ad42c4"
hist_hourly_req = requests.get(url_hist_hourly)
print(f'Status: {hist_hourly_req.status_code}')

wh_hist = hist_hourly_req.json()
print(wh_hist.keys())

wh_hist_df = json_normalize(wh_hist['data'])
wh_hist_df = wh_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg',
       'wind_gust']]
wh_hist_df.head()
'''